In [1]:
# Standard imports + exif tool parser
from os import listdir
import pandas as pd
import exiftool

# Field of view calculation dependencies
from math import degrees, radians, atan
from pyproj import Proj
from camera_calculator import CameraCalculator

# Imports for the UI
import folium
import geopandas as gpd
from shapely.geometry import Polygon
import base64

The below function accepts a single .jpg image, and performs the necessary exif extractions needed for the UI.

In [2]:
def get_exif(image_name="test_images/DJI_0186.JPG"):
    """

    :param image_name: Reference to the .jpg name pulled off the drone
    :return: Single-row dataframe that can be appended to a blank temporary dataframe
    """
    # Grab the metadata for the image
    with exiftool.ExifTool() as et:
        metadata = et.get_metadata(image_name)

    exif_df = pd.DataFrame.from_dict([metadata]) # dict needs to be wrapped in list

    # keep only necessary fields for bbox calculation
    exif_df = exif_df[["File:FileName", "MakerNotes:CameraPitch", "MakerNotes:CameraYaw", "MakerNotes:CameraRoll",
                       "EXIF:GPSAltitude", "EXIF:GPSLatitude", "EXIF:GPSLongitude"]]

    # TODO: MAKE SURE THE SIGNS ON LAT LONG ARE CORRECT
    exif_df["EXIF:GPSLongitude"] = -1 * exif_df["EXIF:GPSLongitude"]

    # Lat Long conversion from decimal degrees to UTM
    # TODO: First, set the UTM zone object
    pp = Proj("+proj=utm +zone=10 +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

    # Convert the centroid lat, long to UTM
    c_x_utm, c_y_utm = pp(exif_df["EXIF:GPSLongitude"].values, exif_df["EXIF:GPSLatitude"].values)
    exif_df["centroid_x_utm"] = c_x_utm
    exif_df["centroid_y_utm"] = c_y_utm - 10000000  # TODO: Figure out why northing gives incorrect value (+10,000,000)

    # DJ Mavic Mini and DJ Mavic Pro sensor specs. These are all fixed values
    img_width_max_px = 4000
    img_height_max_px = 3000
    img_width_cropped = 4000
    img_height_cropped = 2250
    sensor_width_mm = 6.3  # not contained in exif
    sensor_height_mm = 4.7  # not contained in exif
    focal_length = 4.49

    # We need to calculate the horizontal and vertical field of view from the cropped or effective sensor size
    # see: https://mavicpilots.com/threads/mavic-mini-focal-length-versus-field-of-view.85622/
    cropped_sensor_width = img_width_cropped / (img_width_max_px / sensor_width_mm)
    cropped_sensor_height = img_height_cropped / (img_height_max_px / sensor_height_mm)
    fov_h = degrees(atan(cropped_sensor_width / (2 * focal_length))) * 2
    fov_v = degrees(atan(cropped_sensor_height / (2 * focal_length))) * 2

    exif_df["fov_h"] = fov_h
    exif_df["fov_v"] = fov_v

    # Calculate the field of view using the getBoundingPolygon method in camera_calculator.py
    c = CameraCalculator()

    exif_df["bbox_vec"] = exif_df.apply(lambda row: c.getBoundingPolygon(
        radians(row["fov_h"]),
        radians(row["fov_v"]),
        row["EXIF:GPSAltitude"],
        radians(row["MakerNotes:CameraYaw"]),
        radians(row["MakerNotes:CameraRoll"]),
        radians(row["MakerNotes:CameraPitch"]),
        row["centroid_x_utm"],
        row["centroid_y_utm"]
    ), axis=1)

    return exif_df

A single-row dataframe is returned like so:

In [3]:
image_name = "test_images/DJI_0198.JPG"
get_exif(image_name=image_name)

,File:FileName,MakerNotes:CameraPitch,MakerNotes:CameraYaw,MakerNotes:CameraRoll,EXIF:GPSAltitude,EXIF:GPSLatitude,EXIF:GPSLongitude,centroid_x_utm,centroid_y_utm,fov_h,fov_v,bbox_vec
0,DJI_0198.JPG,-89.900002,-0.7,0,42.478,37.362275,-122.07099,582266.795813,4.135467e+06,70.103852,42.863885,[<vector3d.vector.Vector object at 0x127f52940...


Next, we have the build_map() function, which relies on unpack_bbox(). Only build_map() will need to be explicitly called.
It accepts a datframe of the exif images plus cow counts (integer) and writes an .html object to
the current working directory.

In [ ]:
def unpack_bbox(bbox):
    """
    Utility function for build_map(). Takes in vector object from camera_calculator.py
    and converts it into a polygon object for map purposes.
    :param bbox:
    :return: Polygon object which is used to plot on folium map
    """
    x_list = []
    y_list = []
    for i, p in enumerate(bbox):
        x_list.append(p.x)
        y_list.append(p.y)
    # TODO: Ensure correct projection
    pp2 = Proj("+proj=utm +zone=10 +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
    long_list, lat_list = pp2(x_list, y_list, inverse=True)
    bbox_geom = Polygon(zip(long_list, lat_list))
    return bbox_geom

def build_map(exif_df, img_dir="test_images/", sampling_rate=1):
    """
    Call this function to generate the UI (writes .html map object).
    :param exif_df: In-memory dataframe that has the exif data and cow counts
    :param img_dir: Directory where images are stored (probably s3)
    :param sampling_rate: Preserves only every nth image. Used to ensure minimal overlap
    :return:
    """
    # Convert the resultant field of view vector into a valid geometry format for geopandas
    exif_df["bbox_poly_geom"] = exif_df.apply(lambda row: unpack_bbox(row["bbox_vec"]), axis=1)
    gdf = gpd.GeoDataFrame(exif_df, crs="EPSG:4326", geometry=exif_df["bbox_poly_geom"])

    # TODO: Adjust the sampling rate so that we are only counting field of views that have little to no overlap
    filtered_gdf = gdf.iloc[::sampling_rate, :]

    # s3_directory = "https://w251-livestalk-project-test.s3.us-east-2.amazonaws.com/images/" # Used if we're loading a URL

    # Add markers, cow counts, and image thumbnails for every image to the map.
    for c_lat, c_long, img_name, cow_count in zip(filtered_gdf["EXIF:GPSLatitude"],
                                                  filtered_gdf["EXIF:GPSLongitude"],
                                                  filtered_gdf["File:FileName"],
                                                  filtered_gdf["cow_count"]):

        encoded = base64.b64encode(open(img_dir + img_name, 'rb').read())
        html = '<img src="data:image/png;base64,{}">'.format
        iframe = folium.IFrame(html(encoded.decode('UTF-8')), width=400, height=225)
        popup = folium.Popup(iframe, max_width=400)

        # uncomment this instead if you want to just link to the image in the tooltip rather than show on map
        # img_url = s3_directory + img_name
        # popup_url = "<a href=" + img_url+ ">" + str(cow_count) + " cows detected</a>"

        folium.Marker(location=[c_lat, c_long], tooltip="Cow count (Yolov5):"+str(cow_count), popup=popup,
        icon=folium.Icon(color = 'gray')).add_to(map) # set popup=popup_url if you just want to link the thumbnail in s3

    # Writes out html map object into current working directory (can be the same s3 bucket!)
    map.save('livestalk_map_v2.html')

Finally to generate a "live" map, we might be able to set it up by creating a running dataframe
of the exif data and counts, and re-generating the map at some regular interval. Pseudocode below:

In [ ]:
# Create an empty dataframe that will keep a running summary of all image exif data and cow counts
map_df = pd.DataFrame()

# Add new exif rows when in listening mode
while MQTT listens:
    # Extract exif data from the newest image
    map_df = map_df.append(get_exif(new_image.jpg))
    # Insert the yolov5 cow count (int) into a new column
    # Generate latest html object
    build_map(exif_df=map_df, img_dir="test_images/", sampling_rate=1)